# 24강. TF2 API 개요

## 24-3. Tensorflow2 API로 모델 작성하기: MNIST (1) Sequential API 활용

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
 # 흑백이기 때문에 255로 나누어줘서 0~1사이로 정규화

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis] 

print(len(x_train), len(x_test))

11501568/11490434 [==============================] - 0s 0us/step
60000 10000


- np.newaxis는 차원을 바꿔주는 용도. np.reshape은 원소의 개수가 같은 범위 내에서만 차원을 바꿔줄 수 있지만((4,1)->(2,2)는 가능하지만 (3,3)은 불가능) np.newaxis는 그렇지 않다.((4,1)->(3,3) 가능)  
- 그렇다면 여기서는 왜 차원을 바꿔주어야하는걸까???

In [3]:
# Sequential Model을 구성하기
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(32, 3, activation='relu'))
model.add(keras.layers.Conv2D(64, 3, activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [6]:
# 모델 학습
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=2)

model.evaluate(x_test, y_test, verbose=2)
 # verbose : 함수 수행시 발생하는 상세한 정보를 0은 출력하지 않고, 1은 자세히, 2는 함축적인 정보만 출력

Epoch 1/2
1875/1875 [==============================] - 514s 273ms/step - loss: 0.1065 - accuracy: 0.9678
Epoch 2/2
1875/1875 [==============================] - 502s 268ms/step - loss: 0.0346 - accuracy: 0.9888
313/313 - 19s - loss: 0.0457 - accuracy: 0.9853


[0.045698411762714386, 0.9853000044822693]

In [7]:
# IPython의 모든 변수 지우기
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## 24-4. Tensorflow2 API로 모델 작성하기: MNIST (2) Functional API 활용

In [8]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [9]:
# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [12]:
# Functional 모델 구성하기
inputs = keras.Input(shape=(28, 28, 1)) # shape만 넣어주기 때문에 일단 껍데기 만들기(알맹이는 fit하면서)

x = keras.layers.Conv2D(32, 3, activation='relu')(inputs)
x = keras.layers.Conv2D(64, 3, activation='relu')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
predictions = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

In [13]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=2)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/2
1875/1875 [==============================] - 241s 129ms/step - loss: 0.1023 - accuracy: 0.9686
Epoch 2/2
1875/1875 [==============================] - 473s 253ms/step - loss: 0.0332 - accuracy: 0.9896
313/313 - 19s - loss: 0.0393 - accuracy: 0.9861


[0.03933978080749512, 0.9861000180244446]

In [16]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


##  24-5. Tensorflow2 API로 모델 작성하기: MNIST (3) Subclassing 활용

In [17]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [18]:
# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [19]:
# Subclassing을 활용한 Model을 구성하기
class MyModel(keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = keras.layers.Conv2D(32, 3, activation='relu')
        self.conv2 = keras.layers.Conv2D(64, 3, activation='relu')
        self.flatten = keras.layers.Flatten()
        self.d1 = keras.layers.Dense(128, activation='relu')
        self.d2 = keras.layers.Dense(10, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        
        return x
    
model = MyModel()

In [20]:
# 모델 학습 설정
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=2)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/2
1875/1875 [==============================] - 257s 136ms/step - loss: 0.1046 - accuracy: 0.9682
Epoch 2/2
1875/1875 [==============================] - 249s 133ms/step - loss: 0.0338 - accuracy: 0.9892
313/313 - 10s - loss: 0.0485 - accuracy: 0.9849


[0.048542965203523636, 0.9848999977111816]

In [21]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [7]:
# 추가 문제
class A:
    def __init__(self):
        self.apple = 100
        self.banana = 150
        self.orange = 50

class B(A):
    def __init__(self):
        super().__init__()
        
    def call(self, x):
        x = self.apple(x)
        x = self.banana(x)
        x = self.orange(x)
        
        return x

b = B()
b.apple

100

## 24-6. TensorFlow2 API로 모델 작성 및 학습하기: CIFAR-100 (1) Sequential API 활용

In [6]:
import tensorflow as tf
from tensorflow import keras

In [7]:
# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


In [8]:
# Sequential Model을 구성하기
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(16, 3, activation='relu'))
model.add(keras.layers.MaxPool2D((2,2)))
model.add(keras.layers.Conv2D(32, 3, activation='relu'))
model.add(keras.layers.MaxPool2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(100, activation='softmax'))
 # cifar-10의 클래스 개수는 10개, cifar-100의 클래스 개수는 100개

In [9]:
# 모델 학습 설정
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=2)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/2
1563/1563 [==============================] - 46s 29ms/step - loss: 3.6164 - accuracy: 0.1568
Epoch 2/2
1563/1563 [==============================] - 45s 29ms/step - loss: 2.8779 - accuracy: 0.2893
313/313 - 3s - loss: 2.8051 - accuracy: 0.3009


[2.8051037788391113, 0.30090001225471497]

- 커널이 계속 죽어서 따로 학습을 못시켰다.

In [10]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## 24-7. Tensorflow2 API로 모델 작성 및 학습하기: CIFAR-100 (2) Functional API 활용

In [11]:
import tensorflow as tf
from tensorflow import keras

In [12]:
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


In [13]:
# Functional API를 활용한 Model을 구성하기
inputs = keras.Input(shape=(32, 32, 3))

x = keras.layers.Conv2D(16, 3, activation='relu')(inputs)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Conv2D(32, 3, activation='relu')(x)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu')(x)
predictions = keras.layers.Dense(100, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

In [14]:
# 모델 학습 설정
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=2)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/2
1563/1563 [==============================] - 46s 29ms/step - loss: 3.6716 - accuracy: 0.1478
Epoch 2/2
1563/1563 [==============================] - 46s 29ms/step - loss: 2.9814 - accuracy: 0.2691
313/313 - 3s - loss: 2.9674 - accuracy: 0.2709


[2.9673819541931152, 0.27090001106262207]

In [15]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## 24-8. Tensorflow2 API로 모델 작성 및 학습하기: CIFAR-100 (3) Subclassing 활용

In [1]:
import tensorflow
from tensorflow import keras

In [2]:
# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


In [3]:
# Subclassing을 활용한 Model을 구성하기
class MyModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.d1 = keras.layers.Dense(256, activation='relu')
        self.d2 = keras.layers.Dense(100, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        
        return x
    
model = MyModel()

In [4]:
# 모델 학습 설정
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 51s 31ms/step - loss: 3.6243 - accuracy: 0.1556
Epoch 2/5
1563/1563 [==============================] - 48s 31ms/step - loss: 2.9065 - accuracy: 0.2836
Epoch 3/5
1563/1563 [==============================] - 48s 31ms/step - loss: 2.5933 - accuracy: 0.3464
Epoch 4/5
1563/1563 [==============================] - 49s 31ms/step - loss: 2.3791 - accuracy: 0.3909
Epoch 5/5
1563/1563 [==============================] - 48s 31ms/step - loss: 2.2106 - accuracy: 0.4282
313/313 - 3s - loss: 2.5410 - accuracy: 0.3596


[2.5410215854644775, 0.3596000075340271]

## 24-9. GradientTape의 활용

In [1]:
import tensorflow as tf
from tensorflow import keras

# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

# 모델 구성부분
class MyModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.d1 = keras.layers.Dense(256, activation='relu')
        self.d2 = keras.layers.Dense(100, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)

        return x

model = MyModel()
# 여기까지는 subclassing을 활용한 모델 작성법과 동일

50000 10000


In [ ]:
# model.compile(optimizer='adam',  
#              loss='sparse_categorical_crossentropy',  
#              metrics=['accuracy'])  

# 매 스텝 학습이 진행될 때마다 발생하는 loss 및 그래디언트가 
# 어떻게 학습 파라미터를 업데이트하게 되는지를 지정해 주는 작업이 
# model.compile() 안에서 자동으로 진행

In [2]:
# tape.gradient()를 통해 매 스텝 학습이 진행될 때마다 발생하는 그래디언트를 추출한 후 
# optimizer.apply_gradients()를 통해 발생한 그래디언트가 
# 업데이트해야 할 파라미터 model.trainable_variables를 지정해 주는 과정

loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# tf.GradientTape()를 활용한 train_step
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_func(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [4]:
import numpy as np
import time

def train_model(batch_size=32):
    start = time.time()
    for epoch in range(5):
        x_batch = []
        y_batch = []
        for step, (x, y) in enumerate(zip(x_train, y_train)):
            x_batch.append(x)
            y_batch.append(y)
            if step % batch_size == batch_size-1:
                loss = train_step(np.array(x_batch, dtype=np.float32), np.array(y_batch, dtype=np.float32))
                x_batch = []
                y_batch = []
        print('Epoch %d: last batch loss = %.4f' % (epoch, float(loss)))
    print("It took {} seconds".format(time.time() - start))

train_model()

Epoch 0: last batch loss = 3.0630
Epoch 1: last batch loss = 2.6920
Epoch 2: last batch loss = 2.4830
Epoch 3: last batch loss = 2.4134
Epoch 4: last batch loss = 2.3051
It took 318.19452261924744 seconds


train_model() = model.fit()  
train_model()의 경우 한 스텝의 학습 단계(여기서는 train_step 메서드)를 끄집어내서 자유롭게 재구성 가능. 강화학습 또는 GAN(Generative Advasarial Network)의 학습에서는 train_model()을 사용.

In [5]:
# evaluation
prediction = model.predict(x_test, batch_size=x_test.shape[0], verbose=1)
temp = sum(np.squeeze(y_test) == np.argmax(prediction, axis=1))
temp/len(y_test) # accuracy(충분히 학습된 상태가 아니므로 낮음)

1/1 [==============================] - 4s 4s/step


0.3507